# Geopandas - Lab

In [ ]:
%matplotlib inline

import pandas as pd
import geopandas
import shapely.wkt
from shapely.geometry import Point

pd.options.display.max_rows = 10

In [ ]:
import fiona
import rtree

### Importamos los barrios

In [ ]:
barrios = pd.read_csv("barrios.csv", encoding='latin1')

In [ ]:
barrios.head()

### Importamos las comisarías

In [ ]:
comisarias = pd.read_csv("comisarias.csv", encoding = "latin1")

In [ ]:
comisarias

### Creamos los objetos geográficos

Comenzamos generando los objetos geográficos- Para ello construya los GeoDataFrame tanto de los barrios como de las comisarías.

Tenga presente que en el caso de las comisarías será necesario convertir las columnas de caracteres a números, para ello deberá:
- Reemplazar las "," por "."
- Convertir el dtype de object a numeric

### Barrios

In [ ]:
def from_wkt(df, wkt_column):
    
    df["coordinates"]= df[wkt_column].apply(shapely.wkt.loads)
    gdf = geopandas.GeoDataFrame(barrios, geometry='coordinates')
    return gdf

In [ ]:
barrios = from_wkt(barrios, "WKT")

In [ ]:
type(barrios)

### Comisarías

In [ ]:
comisarias["X"] = comisarias["X"].str.replace(",",".").apply(pd.to_numeric)
comisarias["Y"] = comisarias["Y"].str.replace(",",".").apply(pd.to_numeric)

In [ ]:
def from_x_y(df, x, y):
    gdf = geopandas.GeoDataFrame(df.drop([x, y], axis=1),
                                crs={'init': 'epsg:4326'},
                                geometry=[Point(xy) for xy in zip(df[x], df[y])])
    return gdf

In [ ]:
comisarias = from_x_y(comisarias, "X", "Y")

In [ ]:
type(comisarias)

## Visualización

Genere una visualización empleando folium donde veamos las dos capas geográficas.

In [ ]:
import folium
m = folium.Map([-34.606359, -58.443863], zoom_start=12, tiles="OpenStreetMap")
folium.GeoJson(barrios.to_json()).add_to(m)
folium.GeoJson(comisarias.to_json()).add_to(m)
m

¿Hay muchos barrios que no tienen comisarías? ¿Hay barrios que tienen muchas comisarías?

## Spatial joins

Para poder responder esta pregunta necesitamos saber en qué barrio se encuentra cada comisaría. Para ello vamos a realizar un join espacial usando "geopandas.sjoin". Léase la documentación sobre "spatial joins" en el siguiente link:
http://geopandas.org/mergingdata.html

Luego, será necesario sumar la cantidad de comisarías por barrio.

In [ ]:
result=geopandas.sjoin(barrios, comisarias, how = "left")

In [ ]:
print(barrios.crs, comisarias.crs)

In [ ]:
barrios.crs = {'init': 'epsg:4326'}

In [ ]:
result=geopandas.sjoin(barrios, comisarias, how = "left")

In [ ]:
result

In [ ]:
comisarias_barrio = result.groupby("BARRIO").count()["NOMBRE"].sort_values(ascending=False)

In [ ]:
comisarias_barrio.head(10)

In [ ]:
comisarias_barrio.tail(10)

## Áreas lejanas

Vemos varios barrios que no tienen ninguna comisaría, y otros con muchas. Tomemos el centroide de los barrios sin comisarías y veamos a qué distancia se encuentra la comisaría más cercana en cada caso.

Primero, generemos una función que nos permita setear el crs en 4326y luego transforme el mismo a gkba.

In [ ]:
def to_gkba(geodf):
    geodf.crs = {'init' :'epsg:4326'}
    new_geodf = geodf.to_crs(crs = "+proj=tmerc +lat_0=-34.629269 +lon_0=-58.4633 +k=0.9999980000000001 +x_0=100000 +y_0=100000 +ellps=intl +units=m +no_defs")
    return new_geodf

In [ ]:
barrios_gkba = to_gkba(barrios)

In [ ]:
comisarias_gkba = to_gkba(comisarias)

Segundo, generemos una función que dado un polígono, el nombre de un barrio, un conjunto de puntos y un parámetro k, haga un subset y se quede sólo con ese barrio, genere un centroide a partir del polígono y devuelva la distancia de los k puntos más cercanos. 

In [ ]:
def nearest_point(barrio, geodf, points,k):
    geodf = geodf[geodf["BARRIO"]==barrio]
    from shapely.geometry import Point
    centroid = geodf.centroid
    centroid = Point(centroid.geometry.x, centroid.geometry.y)
    distances = [ (i, pt.distance(centroid))  for i, pt in enumerate(points.geometry)]
    dist = [dist for i, dist in distances]
    dist.sort()
    closest = dist[0:k]
    return closest

Apliquemos la función al subconjunto de barrios que no tienen comisarías.

In [ ]:
barrios_sin_comisarias = comisarias_barrio.index[comisarias_barrio==0]

In [ ]:
barrios_sin_comisarias

In [ ]:
[(barrio, nearest_point(barrio,barrios_gkba, comisarias_gkba, 1)) for barrio in barrios_sin_comisarias]